In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
from tqdm.auto import tqdm
from nltk.probability import FreqDist
from nltk import bigrams
from collections import Counter

tqdm.pandas()

In [2]:
text_data = pd.read_csv('articles1.csv')
text_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed      50000 non-null  int64  
 1   id           50000 non-null  int64  
 2   title        50000 non-null  object 
 3   publication  50000 non-null  object 
 4   author       43694 non-null  object 
 5   date         50000 non-null  object 
 6   year         50000 non-null  object 
 7   month        50000 non-null  int64  
 8   url          1 non-null      float64
 9   content      49999 non-null  object 
 10  Unnamed: 10  1 non-null      object 
dtypes: float64(1), int64(3), object(7)
memory usage: 4.2+ MB


In [3]:
text_data.head(10)

,Unnamed,id,title,publication,author,date,year,month,url,content,Unnamed: 10
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016,12,NaN,WASHINGTON — Congressional Republicans have...,NaN
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017,6,NaN,"After the bullet shells get counted, the blood...",NaN
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017,1,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri...",NaN
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017,4,NaN,"Death may be the great equalizer, but it isn’t...",NaN
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017,1,NaN,"SEOUL, South Korea — North Korea’s leader, ...",NaN
5,5,17288,"Sick With a Cold, Queen Elizabeth Misses New Y...",New York Times,Sewell Chan,2017-01-02,2017,1,NaN,"LONDON — Queen Elizabeth II, who has been b...",NaN
6,6,17289,Taiwan’s President Accuses China of Renewed In...,New York Times,Javier C. Hernández,2017-01-02,2017,1,NaN,BEIJING — President Tsai of Taiwan sharpl...,NaN
7,7,17290,"After ‘The Biggest Loser,’ Their Bodies Fought...",New York Times,Gina Kolata,2017-02-08,2017,2,NaN,"Danny Cahill stood, slightly dazed, in a blizz...",NaN
8,8,17291,"First, a Mixtape. Then a Romance. - The New Yo...",New York Times,Katherine Rosman,2016-12-31,2016,12,NaN,"Just how is Hillary Kerr, the founder of ...",NaN
9,9,17292,Calling on Angels While Enduring the Trials of...,New York Times,Andy Newman,2016-12-31,2016,12,NaN,Angels are everywhere in the Muñiz family’s ap...,NaN


In [4]:
for cols in text_data.columns:
    print(cols)

Unnamed
id
title
publication
author
date
year
month
url
content
Unnamed: 10


In [5]:
not_needed = ['id','date','year','month','url','Unnamed','Unnamed: 10', 'publication','author']
text_data = text_data.drop(not_needed, axis = 1)

In [6]:
text_data.head(10)

,title,content
0,House Republicans Fret About Winning Their Hea...,WASHINGTON — Congressional Republicans have...
1,Rift Between Officers and Residents as Killing...,"After the bullet shells get counted, the blood..."
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...","When Walt Disney’s “Bambi” opened in 1942, cri..."
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...","Death may be the great equalizer, but it isn’t..."
4,Kim Jong-un Says North Korea Is Preparing to T...,"SEOUL, South Korea — North Korea’s leader, ..."
5,"Sick With a Cold, Queen Elizabeth Misses New Y...","LONDON — Queen Elizabeth II, who has been b..."
6,Taiwan’s President Accuses China of Renewed In...,BEIJING — President Tsai of Taiwan sharpl...
7,"After ‘The Biggest Loser,’ Their Bodies Fought...","Danny Cahill stood, slightly dazed, in a blizz..."
8,"First, a Mixtape. Then a Romance. - The New Yo...","Just how is Hillary Kerr, the founder of ..."
9,Calling on Angels While Enduring the Trials of...,Angels are everywhere in the Muñiz family’s ap...


In [7]:
test = 'PARIS  —   If the world does not envy the French enough already for their generous vacations, universal health care and fine food and wine, the arrival of 2017 brings this: a newly created “right to disconnect. ” Though ridiculed in some quarters as a ban on   email after hours, it is not quite that. But it is born of the enlightened view that it is actually beneficial for people not to work all the time, and that workers have the right to occasionally draw the line when their employer’s demands intrude on evenings at home, treasured vacations or Sundays with friends and family. “Employees are more and more connected during hours outside of the office,” Myriam El Khomri, the minister of labor, said last year in justifying the need for the law. “The boundary between professional and personal life has become tenuous,” and cases of burnout are becoming more prevalent, she said. The measure is one of a raft of new laws that took effect with the beginning of the new year and that exemplify the search for compromises between preserving French traditions and making concessions to the realities of the modern world. The new provision in the labor law does not ban   emails, but does require that companies with more than 50 employees negotiate a new protocol to ensure that work does not spill into days off or   hours. Some consultants have recommended that employees and managers avoid the “reply all” function on emails to groups so that only one person is being asked to read an email and respond, rather than half the office. Another approach recommends setting a time each evening after which employees are not expected to reply  —   several firms have designated the 10 hours between 9 p. m. and 7 a. m. others the 12 hours between 7 p. m. and 7 a. m. As a country with Catholic roots, but also a commitment to personal liberties, France has had an ambivalent approach to divorce. It has long been legal, but not necessarily speedy. A new nod to modernity in French laws eases the rules for people who want to divorce. While historically France has been far more flexible than Ireland or Italy, it still required a judge to rule on each divorce. The procedure routinely took as long as a year, and sometimes far longer, because cases were backed up awaiting the judicial signature. Now, if both members of the couple agree on the divorce, lawyers can draw up the divorce agreement, jointly sign the papers and have them notarized. No judges need be involved. Smoking is another area in which French habits have changed relatively little in recent years  —   27 to 28 percent of the population still lights up  —   but the country is now following many others in requiring “neutral packaging” for tobacco products: Instead of advertising a brand, cigarette packs sport only health warnings and photos of illnesses caused by smoking. At least two new laws demonstrate the country’s gradual move toward more sustainable products. Instead of thin plastic bags, French supermarkets and fruit vendors must substitute either bags made with a   synthetic called amidon that is mixed with plastic, a thicker type of plastic bag that could be reused, or paper bags. At Monoprix, a supermarket chain, paper bags have won out and stacks of them perch precariously on stands at the ends of fruit counters. More radical is the edict that went into effect on Sunday banning the use of pesticides in public gardens and along public highways. It promises to make public green spaces safer for birds and other small animals, which are especially vulnerable to the poisons used in pest killers. It will not be easy for the gardeners employed by cities to turn to more sustainable methods. When the city of Lyon abandoned pesticides voluntarily nine years ago, it took quite some time to change the culture, although Lyon is now considered a model. In 2019, the antipesticide law will expand to include amateur gardeners  —   a challenge not only for the French with backyard rows of dahlias and daisies, but also for those who nurse roses in their window boxes. A new   law is sure to please the French because it plays to their immense pride in local comestibles: It allows prepared foods, like frozen dinners, to be labeled “produit d’origine française” only if the item is made with 100 percent French meat or milk. Any product with more than 8 percent foreign meat must say where the animal was born, raised and butchered. A product with more than 50 percent foreign milk must say where it was collected and turned into the product being sold. It seems that the French version of “Made in America” is “Raised in France. '

In [8]:
text_data= text_data[~text_data['content'].isna()]

In [9]:
def my_tokenizer_part1(text):
    '''This function is designed to isolate all the tokens from the text and create the tokens'''
    stop_words = set(stopwords.words('english'))
    text =  word_tokenize(text)
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    tokens = [word for word in text if word not in stop_words and word.isalpha() and word not in punctuations] 
    #tokens = [ps.stem(word) for word in tokens]
    final_tokens = [word.lower() for word in tokens]
    return final_tokens
    

In [10]:
def my_tokenizer_part2(text):
    '''This function is designed to isolate all the tokens from the text and create the tokens'''
    stop_words = set(stopwords.words('english'))
    text =  word_tokenize(text)
   
    return text
    

In [11]:
text2 = my_tokenizer_part2(test)
print(text2)

['PARIS', '—', 'If', 'the', 'world', 'does', 'not', 'envy', 'the', 'French', 'enough', 'already', 'for', 'their', 'generous', 'vacations', ',', 'universal', 'health', 'care', 'and', 'fine', 'food', 'and', 'wine', ',', 'the', 'arrival', 'of', '2017', 'brings', 'this', ':', 'a', 'newly', 'created', '“', 'right', 'to', 'disconnect', '.', '”', 'Though', 'ridiculed', 'in', 'some', 'quarters', 'as', 'a', 'ban', 'on', 'email', 'after', 'hours', ',', 'it', 'is', 'not', 'quite', 'that', '.', 'But', 'it', 'is', 'born', 'of', 'the', 'enlightened', 'view', 'that', 'it', 'is', 'actually', 'beneficial', 'for', 'people', 'not', 'to', 'work', 'all', 'the', 'time', ',', 'and', 'that', 'workers', 'have', 'the', 'right', 'to', 'occasionally', 'draw', 'the', 'line', 'when', 'their', 'employer', '’', 's', 'demands', 'intrude', 'on', 'evenings', 'at', 'home', ',', 'treasured', 'vacations', 'or', 'Sundays', 'with', 'friends', 'and', 'family', '.', '“', 'Employees', 'are', 'more', 'and', 'more', 'connected', 

In [ ]:
#text_data['content'] = text_data['content'].progress_apply(my_tokenizer_part1)

In [12]:
text_data['titletoke'] = None

In [13]:
text_data['contenttoke'] =None

In [14]:
text_data['titletoke'] = text_data['title'].progress_apply(my_tokenizer_part2)

  0%|          | 0/49999 [00:00<?, ?it/s]

In [15]:
text_data['contenttoke'] = text_data['content'].progress_apply(my_tokenizer_part2)

  0%|          | 0/49999 [00:00<?, ?it/s]

In [16]:
for col in ['title', 'content']:
    text_data[col] = text_data[col].progress_apply(my_tokenizer_part1)

  0%|          | 0/49999 [00:00<?, ?it/s]

  0%|          | 0/49999 [00:00<?, ?it/s]

In [17]:
text_data.head(10)

,title,content,titletoke,contenttoke
0,"[house, republicans, fret, about, winning, the...","[washington, congressional, republicans, new, ...","[House, Republicans, Fret, About, Winning, The...","[WASHINGTON, —, Congressional, Republicans, ha..."
1,"[rift, between, officers, residents, killings,...","[after, bullet, shells, get, counted, blood, d...","[Rift, Between, Officers, and, Residents, as, ...","[After, the, bullet, shells, get, counted, ,, ..."
2,"[tyrus, wong, bambi, artist, thwarted, racial,...","[when, walt, disney, bambi, opened, critics, p...","[Tyrus, Wong, ,, ‘, Bambi, ’, Artist, Thwarted...","[When, Walt, Disney, ’, s, “, Bambi, ”, opened..."
3,"[among, deaths, heavy, toll, pop, music, the, ...","[death, may, great, equalizer, necessarily, ev...","[Among, Deaths, in, 2016, ,, a, Heavy, Toll, i...","[Death, may, be, the, great, equalizer, ,, but..."
4,"[kim, says, north, korea, is, preparing, test,...","[seoul, south, korea, north, korea, leader, ki...","[Kim, Jong-un, Says, North, Korea, Is, Prepari...","[SEOUL, ,, South, Korea, —, North, Korea, ’, s..."
5,"[sick, with, cold, queen, elizabeth, misses, n...","[london, queen, elizabeth, ii, battling, cold,...","[Sick, With, a, Cold, ,, Queen, Elizabeth, Mis...","[LONDON, —, Queen, Elizabeth, II, ,, who, has,..."
6,"[taiwan, president, accuses, china, renewed, i...","[beijing, president, tsai, taiwan, sharply, cr...","[Taiwan, ’, s, President, Accuses, China, of, ...","[BEIJING, —, President, Tsai, of, Taiwan, shar..."
7,"[after, the, biggest, loser, their, bodies, fo...","[danny, cahill, stood, slightly, dazed, blizza...","[After, ‘, The, Biggest, Loser, ,, ’, Their, B...","[Danny, Cahill, stood, ,, slightly, dazed, ,, ..."
8,"[first, mixtape, then, romance, the, new, york...","[just, hillary, kerr, founder, digital, media,...","[First, ,, a, Mixtape, ., Then, a, Romance, .,...","[Just, how, is, Hillary, Kerr, ,, the, founder..."
9,"[calling, angels, while, enduring, trials, job...","[angels, everywhere, muñiz, family, apartment,...","[Calling, on, Angels, While, Enduring, the, Tr...","[Angels, are, everywhere, in, the, Muñiz, fami..."


In [18]:
all_tokens = []
all_filt_tokens = []
for i in range(0,49999):
    all_filt_tokens += text_data.iloc[i,0] + text_data.iloc[i,1]
    all_tokens += text_data.iloc[i,2]+text_data.iloc[i,3]
    

In [19]:
def lex_calc(all, uniq):
    '''This guy will calculate lex of any two tokens'''
    a = len(all)
    b = len(uniq)
    result = b/a
    return result

In [25]:
all_tokens

['House',
 'Republicans',
 'Fret',
 'About',
 'Winning',
 'Their',
 'Health',
 'Care',
 'Suit',
 '-',
 'The',
 'New',
 'York',
 'Times',
 'WASHINGTON',
 '—',
 'Congressional',
 'Republicans',
 'have',
 'a',
 'new',
 'fear',
 'when',
 'it',
 'comes',
 'to',
 'their',
 'health',
 'care',
 'lawsuit',
 'against',
 'the',
 'Obama',
 'administration',
 ':',
 'They',
 'might',
 'win',
 '.',
 'The',
 'incoming',
 'Trump',
 'administration',
 'could',
 'choose',
 'to',
 'no',
 'longer',
 'defend',
 'the',
 'executive',
 'branch',
 'against',
 'the',
 'suit',
 ',',
 'which',
 'challenges',
 'the',
 'administration',
 '’',
 's',
 'authority',
 'to',
 'spend',
 'billions',
 'of',
 'dollars',
 'on',
 'health',
 'insurance',
 'subsidies',
 'for',
 'and',
 'Americans',
 ',',
 'handing',
 'House',
 'Republicans',
 'a',
 'big',
 'victory',
 'on',
 'issues',
 '.',
 'But',
 'a',
 'sudden',
 'loss',
 'of',
 'the',
 'disputed',
 'subsidies',
 'could',
 'conceivably',
 'cause',
 'the',
 'health',
 'care',
 

In [20]:
num_tokens = len(all_tokens)

In [21]:
num_tokens

38856641

In [22]:
unique_tokens = set(all_tokens)

In [23]:
len(unique_tokens)

235321

In [24]:
lex_calc(all_tokens, unique_tokens)

0.006056133364693052

In [26]:
fre_dis = FreqDist(all_tokens)
sorted_fdist = sorted(fre_dis.items(), key=lambda x: x[1], reverse = True)

In [27]:
sorted_fdist[:3]

[(',', 1869183), ('the', 1669196), ('.', 1456655)]

In [24]:
num_types = len(unique_tokens)

In [25]:
num_types

166103

In [26]:
lex_d = num_types / num_tokens

In [27]:
lex_d

0.008775333695716547

In [48]:
len(filtered_toke)

17685972

In [28]:
frequency_distribution = FreqDist(all_tokens)

In [30]:
sorted_fdist = sorted(frequency_distribution.items(), key=lambda x: x[1], reverse = True)

In [41]:
top4 = sorted_fdist[:4]

In [42]:
top4

[('the', 222123), ('said', 207562), ('i', 171601), ('trump', 159667)]

In [33]:
one_fdist = {}
for key,value in frequency_distribution.items():
    if value == 1:
        one_fdist[key] = value

In [35]:
len(one_fdist)

60955

In [49]:
stop_words = set(stopwords.words('english'))
filtered_toke = [word for word in all_tokens if word not in stop_words]

In [61]:
fd = FreqDist(filtered_toke)

In [65]:
sorted_f = sorted(fd.items(), key=lambda x: x[1], reverse = True)

In [67]:
top3 = sorted_f[:4]

In [68]:
top3

[('said', 207562), ('trump', 159667), ('people', 77905), ('one', 77357)]

In [58]:
uniq_filter = set(filtered_toke)

In [59]:
lex_div_withnostop = lex_calc(filtered_toke,uniq_filter)

In [60]:
lex_div_withnostop

0.009383199294898805

In [29]:
len(all_filt_tokens)

18928397

In [31]:
bi_grams = list(bigrams(all_filt_tokens))

In [35]:
bigram_counter = Counter(bi_grams)
most_frequent_bigrams = bigram_counter.most_common(3)

In [36]:
most_frequent_bigrams

[(('donald', 'trump'), 29245),
 (('new', 'york'), 27196),
 (('united', 'states'), 22857)]